In [3]:
import pygame as pg
import pymunk
import numpy as np
import random

random.seed(0)

pygame 2.1.0 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [96]:
# docs
# pygame for visualization: https://www.pygame.org/docs/
# pymunk as physics engine: https://www.pymunk.org/en/latest/overview.html

# references
# simulator with pymunk: https://www.youtube.com/watch?v=yJK5J8a7NFs (used this one a lot)
# pygame city builder: https://www.youtube.com/watch?v=wI_pvfwcPgQ
# james allen - agent based modelling: https://www.youtube.com/watch?v=RglNX4c_dfc

In [17]:
# report

# explain our particles (persons): 
#    - shape (with attributes)
#    - velocity
#    - collision_radius

In [4]:
# constants
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
LIGHT_GREY = (70, 84, 105)

FPS = 60

In [10]:
# Playground
a = np.ndarray((10,10))
a[0]

array([4.67055536e-310, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000])

In [ ]:
class Person():
    
    def __init__(self, world, x_init, y_init, collision_radius=10):
        
        # setup person as a pymunk circle particle
        self.x = x_init
        self.y = y_init
        self.collision_radius = collision_radius
        self.body = pymunk.Body()
        self.body.position = x_init, y_init
        
        # initial velocity
        self.body.velocity = random.uniform(-100, 100), random.uniform(-100, 100)
        # self.body.velocity = 50, 50
        # self.body.velocity = self.x/10, self.y/10
        
        self.shape = pymunk.Circle(self.body, self.collision_radius)
        self.shape.density = 1
        self.shape.elasticity = 1
        
        # setup attributes
        # job
        # age
        # vaccinated
        # planned_path
        # etc.
        
        # add the person to the world
        world.add(self.body, self.shape)
    
    def update_velocity(self):
        # make dependent on current velocity (self.body.velocity) and planned path
        pass
    
    def draw(self, screen):
        x, y = self.body.position
        discrete_position = (int(x), int(y))
        pg.draw.circle(screen, BLACK, discrete_position, self.collision_radius)

In [19]:
class Wall():
    def __init__(self, world, start_pos, end_pos, thickness=3):
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.thickness = thickness
        self.body = pymunk.Body(body_type=pymunk.Body.STATIC) # static body
        self.shape = pymunk.Segment(self.body, start_pos, end_pos, radius=thickness) # people might glitch through if not big enough
        self.shape.elasticity = 1
        world.add(self.body, self.shape)
    
    def draw(self, screen):
        pg.draw.line(screen, LIGHT_GREY, self.start_pos, self.end_pos, self.thickness)

In [20]:
class VirusSim():
    
    def __init__(self, n_people, screen_size, world_size):
        # game setup
        self.screen = pg.display.set_mode((screen_size, screen_size))
        self.width, self.height = self.screen.get_size()
        self.clock = pg.time.Clock()
        self.running = True
        
        # logo and caption
        logo = pg.image.load('images/virus_logo.png')
        pg.display.set_icon(logo)
        pg.display.set_caption('Virus Sim')
        
        # setup world (one tile is a 10px by 10px square)
        self.world = pymunk.Space()
        self.world_size = world_size
        
        # add screen borders as walls
        self.borders = [
            Wall(world=self.world, start_pos=(0, 0), end_pos=(800, 0)),
            Wall(world=self.world, start_pos=(0, 0), end_pos=(0, 800)),
            Wall(world=self.world, start_pos=(0, 800), end_pos=(800, 800)),
            Wall(world=self.world, start_pos=(800, 0), end_pos=(800, 800))]
        
        # add more walls
        house1, bg1 = self._create_tile(origin_pos=(0,0), tile_type='house')
        house2, bg2 = self._create_tile(origin_pos=(80,0), tile_type='house')
        house3, bg3 = self._create_tile(origin_pos=(160,0), tile_type='house')
        house4, bg4 = self._create_tile(origin_pos=(240,80), tile_type='house')
        
        # add all tiles to a list
        self.tiles = [house1, house2, house3, house4]
        self.backgrounds = [bg1, bg2, bg3, bg4]
        
        # create people
        self.n_people = n_people
        self.people = [Person(world=self.world,
                              x_init=random.randint(0, screen_size),
                              y_init=random.randint(0, screen_size),
                              collision_radius=2)
                       for i in range(self.n_people)]
        
        # setup np-arrays for data
        
    def _create_tile(self, origin_pos, tile_type):
        if tile_type == 'house':
            x, y = origin_pos
            tile_walls = [
                # create main walls
                Wall(world=self.world, start_pos=(x, y), end_pos=(x+80, y)),
                Wall(world=self.world, start_pos=(x, y), end_pos=(x, y+80)),
                Wall(world=self.world, start_pos=(x+80, y), end_pos=(x+80, y+80)),
                
                # create half-open wall
                Wall(world=self.world, start_pos=(x, y+80), end_pos=(x+20, y+80)),
                Wall(world=self.world, start_pos=(x+60, y+80), end_pos=(x+80, y+80))]
            
            # create background images
            tile_bg_img = pg.image.load('images/house2d.png')
            tile_bg_img.set_alpha(130)
            tile_bg_img = pg.transform.scale(tile_bg_img, (80, 80))
        
        background = tile_bg_img, origin_pos
        return tile_walls, background
    
    def run(self):
        while self.running:
            
            self.clock.tick(FPS) # fps
            self.world.step(1/FPS)
            
            self.events()            
            self.update()
            if self.running: # and self.render (if rendering is too slow)
                self.draw()
                            
            # save logs
            # evaluate later
    
    def events(self):
        for event in pg.event.get():
            
            if event.type == pg.QUIT:
                pg.quit()
                self.running = False
                
            if event.type == pg.KEYDOWN:
                if event.key == pg.K_ESCAPE:
                    pg.quit()
                    self.running = False
    
    def update(self):
        pass
    
    def draw(self):
        # fill the background color
        self.screen.fill(WHITE)
        
        # draw background images
        for bg_img, origin_pos in self.backgrounds:
            self.screen.blit(bg_img, origin_pos)
        
        # draw people
        for person in self.people:
            person.draw(self.screen)
        
        # draw borders
        for border in self.borders:
            border.draw(self.screen)
        
        # draw tiles
        for tile in self.tiles:
            for wall in tile:
                wall.draw(self.screen)
        
        pg.display.flip() # update entire screen

In [21]:
pg.init()
sim = VirusSim(n_people=1000, screen_size=800, world_size=80)
sim.run()